Unit testing of handwritten digit recognizer

[Текст ссылки](https://)Перво-наперво внесем в ноутбук исходные функции.

In [ ]:
!pip install tensorFlow

     |███████████████▋                | 221.6 MB 79.4 MB/s eta 0:00:03    |██████▏                         | 88.3 MB 51.6 MB/s eta 0:00:08        | 191.5 MB 73.0 MB/s eta 0:00:04

In [10]:
from tensorflow import keras
 
from keras.datasets import mnist
 
from keras.models import Sequential
 
from keras.layers import Dense, Dropout, Flatten
 
from keras.layers import Conv2D, MaxPooling2D
 
# разделяем данные на тренировку и тестовые наборы
 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
 
print(x_train.shape, y_train.shape)
 
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
 
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
 
input_shape = (28, 28, 1)
 
# преобразуем веркторы классов в бинарные матрицы
 
y_train = keras.utils.to_categorical(y_train, 10)
 
y_test = keras.utils.to_categorical(y_test, 10)
 
x_train = x_train.astype('float32')
 
x_test = x_test.astype('float32')
 
x_train /= 255
 
x_test /= 255
 
print('x_train shape:', x_train.shape)
 
print(x_train.shape[0], 'тренировочные примеры')
 
print(x_test.shape[0], 'тестовые примеры')
 
batch_size = 128
 
num_classes = 10
 
epochs = 10
 
model = Sequential()
 
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
 
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (3, 3), activation='relu'))
 
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
 
model.add(Dense(128, activation='relu'))
 
model.add(Dropout(0.3))
 
model.add(Dense(64, activation='relu'))
 
model.add(Dropout(0.5))
 
model.add(Dense(num_classes, activation='softmax'))
 
model.compile(loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
 
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
 
print("Модель успешно получена")
 
score = model.evaluate(x_test, y_test, verbose=0)
 
print('Тестовые потери:', score[0])
 
print('Тестовая точность:', score[1])
 
model.save('mnist.h5')
 
print("Сохранение модели как mnist.h5")

(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 тренировочные примеры
10000 тестовые примеры
Epoch 1/10
469/469 [==============================] - 94s 195ms/step - loss: 0.7388 - accuracy: 0.7560 - val_loss: 0.0690 - val_accuracy: 0.9779
Epoch 2/10
469/469 [==============================] - 88s 189ms/step - loss: 0.1121 - accuracy: 0.9699 - val_loss: 0.0468 - val_accuracy: 0.9858
Epoch 3/10
469/469 [==============================] - 83s 176ms/step - loss: 0.0770 - accuracy: 0.9802 - val_loss: 0.0428 - val_accuracy: 0.9871
Epoch 4/10
469/469 [==============================] - 84s 179ms/step - loss: 0.0592 - accuracy: 0.9854 - val_loss: 0.0325 - val_accuracy: 0.9907
Epoch 5/10
469/469 [==============================] - 82s 174ms/step - loss: 0.0468 - accuracy: 0.9884 - val_loss: 0.0314 - val_accuracy: 0.9923
Epoch 6/10
469/469 [==============================] - 83s 177ms/step - loss: 0.0438 - accuracy: 0.9890 - val_loss: 0.0375 - val_accuracy: 0.9912
Epoch 7/10
469/469 [

In [19]:
from ipywidgets import Image

from ipywidgets import link, AppLayout, GridspecLayout 

from ipycanvas import Canvas, hold_canvas

import ipywidgets as widgets

from tensorflow import keras

from keras.models import load_model
 
from PIL import ImageGrab, Image
 
import numpy as np

def calculate_res(w, h, k):
    if w > 300:
        w = w/k
    else: w = 300
    if h > 300:
        h = h/k
    else: h = 300
    return (w, h)

model = load_model('mnist.h5')
 
def predict_digit(img):
 
    # ужимаем изображение до 28x28 пикселей
 
    img = img.resize((28, 28))
 
    # преобразуем цвет в градации серого
 
    img = img.convert('L')
 
    img = np.array(img)
 
    # решейпим для поддержки входа модели и нормализации
 
    img = img.reshape(1, 28, 28, 1)
 
    img = img / 255.0
 
    # предсказываем класс
 
    res = model.predict([img])[0]
 
    return np.argmax(res), max(res)
 
def clear_all(arg):
        canvas.clear()
 
def classify_handwriting(arg):
 
        rect = (0 + 4, 0 + 4, 300 - 4, 300 - 4)
 
        image = Image.fromarray(canvas.get_image_data(), 'RGB')
 
        # предсказываем число и точность такого предсказания
 
        digit, accuracy = predict_digit(image)
 
        label.value = str(digit) + ', ' + str(int(accuracy * 100)) + '%'
    
def on_mouse_down(x, y):
        global drawing
        drawing = True
        canvas.fill_circle(x,y,8)

def on_mouse_move(x, y):
        global drawing
        if not drawing:
            return
        canvas.fill_circle(x,y,8)


def on_mouse_up(x, y):
      global drawing
      drawing = False

canvas = Canvas(width = 300, height = 300)
canvas.sync_image_data = True
label = widgets.Label("Draw...")
classify_btn = widgets.Button(
            description='Опознать',
            disabled=False,
            button_style='', 
            tooltip='recognize digit'
        )
classify_btn.on_click(classify_handwriting)
 
button_clear = widgets.Button(
            description='Очистить',
            disabled=False,
            button_style='', 
            tooltip='clear digit'
        )
button_clear.on_click(clear_all)
        
        
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = '#000000'
canvas.fill_style = '#000000'
        
grid = GridspecLayout(4, 3, height='300px')

grid[:3, 1:] = label
grid[:, 0] = canvas
grid[3, 1] = button_clear
grid[3, 2] = classify_btn

grid

GridspecLayout(children=(Label(value='Draw...', layout=Layout(grid_area='widget001')), Canvas(height=300, layo…

In [18]:
import unittest

class TestNotebook(unittest.TestCase):
    def test_calculating_res(self):
        (w1, h1) = calculate_res(900, 900, 3)
        self.assertEqual(w1, 300)
        self.assertEqual(h1, 300)
        
        (w2, h2) = calculate_res(1, 1, 3)
        self.assertEqual(w2, 300)
        self.assertEqual(h2, 300)
        
        (w3, h3) = calculate_res(9000, 9000, 3)
        self.assertEqual(w2, 300)

unittest.main(argv=[''], verbosity=2, exit=False)

test_calculating_res (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
